In [1]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver import ActionChains
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.select import Select

from bs4 import BeautifulSoup
from urllib.request import urlopen
import urllib
from urllib.request import Request
import requests

import time
from tqdm.notebook import tqdm

import re
import pandas as pd
import numpy as np

from fake_useragent import UserAgent
import ssl

### 국내 보드게임 커뮤니티 사이트 보드라이프 크롤링

In [70]:
url = 'https://boardlife.co.kr/rank.php?pg={page_num}&type=&number='

name = []
link=[]
rank=[]
bl_grade=[]
user_grade=[]

for a in tqdm(range(1,57)) : 
    url1 = url.format(page_num = urllib.parse.quote(str(a)))
    html = urlopen(url1)
    soup = BeautifulSoup(html, "html.parser")
    
    info_box = soup.find(class_ = 'game-rank-wrapper').find_all(class_ = 'game-rank-row-wrapper')
    if float(info_box[0].find(class_ = 'storage-avg').string )== 0 :
        break
    
    for i in info_box :
        if float(i.find(class_ = 'storage-avg').string) == 0 :
            break
        name.append(i.find(class_ = 'storage-sub-title').string)
        rank.append(i.find(class_ = 'storage-rank').string)
        link.append(i["href"])
        bl_grade.append(i.find(class_ = 'storage-avg').string)
        user_grade.append(i.find(class_ = 'storage-point').string)
        
    

  0%|          | 0/56 [00:00<?, ?it/s]

In [71]:
len(name)

5559

In [72]:
bl_grade[-1]

'5.879'

In [75]:
geek_link = []
url2 = "https://boardlife.co.kr/"


b = 0

for i in tqdm(link[:100]) :

    url3 = url2 + i + '&load=&collect='
    html = urlopen(url3)
    soup = BeautifulSoup(html, "html.parser")
    
    try :
        a = soup.find_all(class_ ='game-intro-div-title')[-1]
        geek_link.append(a.find('a')['href'])
    except :
        geek_link.append(np.nan)
    b+=1

  0%|          | 0/100 [00:00<?, ?it/s]

In [76]:
len(geek_link)

100

### 상위 100개 게임 비교하기 위한 데이터 프레임

In [78]:
boardlife_top100_df = pd.DataFrame({
    "이름" : name[:100],
    "geek링크" : geek_link[:100],
    "랭크" : rank[:100],
    "bl평점" : bl_grade[:100],
    "유저평점" : user_grade[:100],
    "상세정보링크" : link[:100]
})
boardlife_top100_df.head(20)

,이름,geek링크,랭크,bl평점,유저평점,상세정보링크
0,Gloomhaven,https://www.boardgamegeek.com/boardgame/174430...,1,8.564,9.23,bbs_detail.php?bbs_num=5625&tb=boardgame_strategy
1,Terraforming Mars,https://www.boardgamegeek.com/boardgame/167791...,2,8.231,8.39,bbs_detail.php?bbs_num=6501&tb=boardgame_strategy
2,Mage Knight,https://www.boardgamegeek.com/boardgame/96848/...,3,8.237,8.70,bbs_detail.php?bbs_num=519&tb=boardgame_strategy
3,Gaia Project,https://boardgamegeek.com/boardgame/220308/gai...,4,8.326,8.74,bbs_detail.php?bbs_num=9210&tb=boardgame_strategy
4,Pandemic Legacy: Season 1,https://www.boardgamegeek.com/boardgame/161936...,5,8.313,8.74,bbs_detail.php?bbs_num=5572&tb=boardgame_strategy
5,Through the Ages: A New Story of Civilization,https://www.boardgamegeek.com/boardgame/182028...,6,8.318,8.73,bbs_detail.php?bbs_num=5660&tb=boardgame_strategy
6,Brass: Birmingham,https://boardgamegeek.com/boardgame/224517/bra...,7,8.322,8.82,bbs_detail.php?bbs_num=8569&tb=boardgame_strategy
7,Twilight Struggle,https://www.boardgamegeek.com/boardgame/12333/...,8,8.194,8.47,bbs_detail.php?bbs_num=348&tb=boardgame_strategy
8,7 Wonders: Duel,https://www.boardgamegeek.com/boardgame/173346...,9,8.196,8.36,bbs_detail.php?bbs_num=5222&tb=boardgame_strategy
9,Puerto Rico,https://www.boardgamegeek.com/boardgame/3076/p...,10,8.206,8.39,bbs_detail.php?bbs_num=90&tb=boardgame_strategy


In [79]:
boardlife_top100_df['geek링크'] = boardlife_top100_df['geek링크'].str.replace('https://www.boardgamegeek.com','')
boardlife_top100_df['geek링크'] = boardlife_top100_df['geek링크'].str.replace('https://boardgamegeek.com','')
boardlife_top100_df['geek링크'] = boardlife_top100_df['geek링크'].str.replace('http://boardgamegeek.com','')
boardlife_top100_df['geek링크']

/tmp/ipykernel_20648/1269332575.py:1: FutureWarning: The default value of regex will change from True to False in a future version.
  boardlife_top100_df['geek링크'] = boardlife_top100_df['geek링크'].str.replace('https://www.boardgamegeek.com','')
/tmp/ipykernel_20648/1269332575.py:2: FutureWarning: The default value of regex will change from True to False in a future version.
  boardlife_top100_df['geek링크'] = boardlife_top100_df['geek링크'].str.replace('https://boardgamegeek.com','')
/tmp/ipykernel_20648/1269332575.py:3: FutureWarning: The default value of regex will change from True to False in a future version.
  boardlife_top100_df['geek링크'] = boardlife_top100_df['geek링크'].str.replace('http://boardgamegeek.com','')


0                          /boardgame/174430/gloomhaven
1                   /boardgame/167791/terraforming-mars
2               /boardgame/96848/mage-knight-board-game
3                        /boardgame/220308/gaia-project
4                     /boardgame/161936/pandemic-legacy
                            ...                        
95    /boardgame/104162/descent-journeys-dark-second...
96                                 /boardgame/5/acquire
97                                  /boardgame/13/catan
98    /boardgame/103343/game-thrones-board-game-seco...
99                                  /boardgame/54/tikal
Name: geek링크, Length: 100, dtype: object

In [80]:
boardlife_top100_df.head(20)

,이름,geek링크,랭크,bl평점,유저평점,상세정보링크
0,Gloomhaven,/boardgame/174430/gloomhaven,1,8.564,9.23,bbs_detail.php?bbs_num=5625&tb=boardgame_strategy
1,Terraforming Mars,/boardgame/167791/terraforming-mars,2,8.231,8.39,bbs_detail.php?bbs_num=6501&tb=boardgame_strategy
2,Mage Knight,/boardgame/96848/mage-knight-board-game,3,8.237,8.70,bbs_detail.php?bbs_num=519&tb=boardgame_strategy
3,Gaia Project,/boardgame/220308/gaia-project,4,8.326,8.74,bbs_detail.php?bbs_num=9210&tb=boardgame_strategy
4,Pandemic Legacy: Season 1,/boardgame/161936/pandemic-legacy,5,8.313,8.74,bbs_detail.php?bbs_num=5572&tb=boardgame_strategy
5,Through the Ages: A New Story of Civilization,/boardgame/182028/through-ages-new-story-civil...,6,8.318,8.73,bbs_detail.php?bbs_num=5660&tb=boardgame_strategy
6,Brass: Birmingham,/boardgame/224517/brass-birmingham,7,8.322,8.82,bbs_detail.php?bbs_num=8569&tb=boardgame_strategy
7,Twilight Struggle,/boardgame/12333/twilight-struggle,8,8.194,8.47,bbs_detail.php?bbs_num=348&tb=boardgame_strategy
8,7 Wonders: Duel,/boardgame/173346/7-wonders-duel,9,8.196,8.36,bbs_detail.php?bbs_num=5222&tb=boardgame_strategy
9,Puerto Rico,/boardgame/3076/puerto-rico,10,8.206,8.39,bbs_detail.php?bbs_num=90&tb=boardgame_strategy


In [3]:
boardgame_df_result = pd.read_csv('../data/boardgame_df.csv', encoding='utf-8') 
boardgame_df_result

,이름,출시일,url,geek평점,평균평점,가격,랭크,최소인원,최대인원,최소게임시간,최대게임시간,난이도,진행방식,종류,최소나이,최대나이
0,Gloomhaven,2017.0,/boardgame/174430/gloomhaven,8.441,8.67,105.98,1,1,4,60,120,3.888553,"Action Queue, Action Retrieval, Campaign / Bat...","Adventure, Exploration, Fantasy, Fighting, Min...",14.0,NaN
1,Brass: Birmingham,2018.0,/boardgame/224517/brass-birmingham,8.434,8.65,247.97,2,2,4,60,120,3.897451,"Hand Management, Income, Loans, Market, Networ...","Economic, Industry / Manufacturing, Post-Napol...",14.0,NaN
2,Pandemic Legacy: Season 1,2015.0,/boardgame/161936/pandemic-legacy-season-1,8.411,8.57,68.21,3,2,4,60,60,2.832465,"Action Points, Cooperative Game, Hand Manageme...","Environmental, Medical",12.0,NaN
3,Ark Nova,2021.0,/boardgame/342942/ark-nova,8.291,8.57,62.99,4,1,4,90,150,3.710414,"End Game Bonuses, Hand Management, Hexagon Gri...","Animals, Economic, Environmental",12.0,NaN
4,Twilight Imperium: Fourth Edition,2017.0,/boardgame/233078/twilight-imperium-fourth-edi...,8.251,8.64,121.43,5,3,6,240,480,4.293668,"Action Drafting, Area Majority / Influence, Ar...","Civilization, Economic, Exploration, Negotiati...",14.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23851,LCR,1983.0,/boardgame/3522/lcr,4.426,3.54,7.58,23896,3,12,20,20,1.045802,Dice Rolling,Dice,4.0,NaN
23852,The Game of Life,1960.0,/boardgame/2921/game-of-life,4.334,4.33,21.99,23897,2,6,60,60,1.169468,"Events, Roll / Spin and Move, Simulation","Childrens Game, Economic, Electronic",6.0,18.0
23853,Trouble,1965.0,/boardgame/1410/trouble,4.314,3.82,10.52,23898,2,4,45,45,1.069672,"Race, Roll / Spin and Move",Childrens Game,4.0,14.0
23854,Monopoly,1935.0,/boardgame/1406/monopoly,4.292,4.36,23.95,23899,2,8,60,180,1.630500,"Auction/Bidding, Auction: English, Income, Loa...","Economic, Negotiation",8.0,NaN


### 보드라이프와 boardgamegeek 비교
- 두 데이터를 비교하기 위해 이름을 통일시키는 과정

In [82]:
for i in tqdm(range(len(boardlife_top100_df['geek링크']))) :
    for j in range(len(boardgame_df_result['url'])) :
        if boardlife_top100_df['geek링크'][i] == boardgame_df_result['url'][j] :
            boardlife_top100_df['이름'][i] = boardgame_df_result['이름'][j]

            print(boardlife_df['이름'][i])
            print(boardgame_df_result['이름'][j])
            break
    

  0%|          | 0/100 [00:00<?, ?it/s]

Gloomhaven
Gloomhaven
Terraforming Mars
Terraforming Mars
Mage Knight Board Game
Mage Knight Board Game
Gaia Project
Gaia Project
Through the Ages: A New Story of Civilization
Through the Ages: A New Story of Civilization
Brass: Birmingham
Brass: Birmingham
Twilight Struggle
Twilight Struggle
7 Wonders Duel
7 Wonders Duel
Puerto Rico
Puerto Rico
Agricola
Agricola
Arkham Horror: The Card Game
Arkham Horror: The Card Game
The Castles of Burgundy
The Castles of Burgundy
Barrage
Barrage
Dominion
Dominion
Spirit Island
Spirit Island
The Gallerist
The Gallerist
Android: Netrunner
Android: Netrunner
Tzolk'in: The Mayan Calendar
Tzolk'in: The Mayan Calendar
Le Havre
Le Havre
Grand Austria Hotel
Grand Austria Hotel
Terra Mystica
Terra Mystica
Ark Nova
Ark Nova
The Resistance: Avalon
The Resistance: Avalon
Splendor
Splendor
Tichu
Tichu
Skull King
Skull King
Lorenzo il Magnifico
Lorenzo il Magnifico
Scythe
Scythe
Caverna: The Cave Farmers
Caverna: The Cave Farmers
Trajan
Trajan
Star Wars: Rebelli

In [84]:
v = 0
for i in range(100) :
    if boardlife_top100_df['이름'][i] in list(boardgame_df_result['이름'][:100]) :
        v+=1
    else :
        print(i)
        print(boardlife_top100_df['이름'][i])
        print('-------------')
        
print(v)

13
Mage Knight: Ultimate Edition
-------------
14
Terra Mystica Big Box
-------------
16
Dominion
-------------
26
The Resistance: Avalon
-------------
27
Splendor
-------------
28
Brass
-------------
29
Tichu
-------------
30
Skull King
-------------
31
Lorenzo il Magnifico
-------------
34
Terraforming Mars: Prelude
-------------
35
Trajan
-------------
38
Las Vegas
-------------
39
Telestrations
-------------
45
Yokohama
-------------
46
Eldritch Horror
-------------
47
Age of Steam
-------------
48
Robinson Crusoe: Adventure on the Cursed Island
-------------
49
Dead of Winter: A Crossroads Game
-------------
51
The Castles of Burgundy (with expansions)
-------------
60
Vinhos: Deluxe Edition
-------------
61
Alchemists
-------------
66
Mahjong
-------------
69
新幕 桜降る代に決闘を 基本セット (A Duel for Falling Cherry Blossoms: New Act – Basic Set)
-------------
70
Modern Art
-------------
71
Skull King
-------------
72
Seasons
-------------
74
Russian Railroads
-------------
75
At the Gates of

In [85]:
# 위의 코드로 통일시키지 못한 부분 통일
boardlife_top100_df['이름'][28] = boardgame_df_result['이름'][19]
boardlife_top100_df['geek링크'][28] = boardgame_df_result['url'][19]

In [87]:
boardlife_top100_df['이름'][48] = boardgame_df_result['이름'][80]
boardlife_top100_df['geek링크'][48] = boardgame_df_result['url'][80]

In [89]:
boardlife_top100_df.to_csv('../data/boardlife_top100_df.csv', sep = "," , encoding = "utf-8",index = False)

In [80]:
boardlife_name_dup = list(set(boardlife_top100_df['이름']))

In [81]:
len(boardlife_name_dup)

97

In [82]:
count = 0
for i in range(len(boardlife_name_dup)) :
    if boardlife_name_dup[i] in list(boardgame_df_result['이름'][:100]) :
        count+=1
        
print(count)

60


In [83]:
#보드라이프 상위 100개 게임중 boardgamegeek에 없는 게임 수

n_count = 0
for i in tqdm(range(len(boardlife_name_dup))) :
    if boardlife_name_dup[i] not in list(boardgame_df_result['이름']) :
        n_count += 1

print(n_count)

  0%|          | 0/97 [00:00<?, ?it/s]

4


상위 100위 안에 있는 게임중 60개가 동일하다.  
보드라이프에서 랭킹 안에 있는 게임수는 5559개, boardgamegeek 랭킹 안에 있는 게임수는 23900개다.  
-> 기재된 게임 수 차이가 크다.
한국 보드라이프 커뮤니티 게임랭킹에 같은 게임이 2번 들어가 있는 경우도 있고  
boardgamegeek 랭킹 경우에는 원본 게임들의 확장팩은 뺐기 때문에 실질적인 비교 개수는 93개다.
더군다나 보드라이프 상위 93개 게임중 위의 60개의 게임을 제외한 33개의 게임도 boardgamegeek 랭킹의 상위 3%안에 든다.  
그러므로 한국에서 인기있는 보드게임과 해외에서 인기있는 보드게임의 성향이 비슷하다고 볼 수 있다.


